In [172]:
import requests,time,re,time,itertools,csv,json,pprint,pandas as pd
from pyquery import PyQuery as pq
proxy_host = "proxy.crawlera.com"
proxy_port = "8010"
proxy_auth = "f004a7eade8142c4bd96cd80a74ab9fd:" # Make sure to include ':' at the end
proxies = {"https": "https://{}@{}:{}/".format(proxy_auth, proxy_host, proxy_port),
          "http": "http://{}@{}:{}/".format(proxy_auth, proxy_host, proxy_port)}


#define a function to replace all strings at the same time 
#reference:https://blog.csdn.net/kwame211/article/details/88972083
def one_rel(text,d):
    pattern=re.compile('|'.join([re.escape(x) for x in d.keys()]))
    def return_func(match):  
        return d[match.group(0)]
    is_replaced=True
    if pattern.sub(return_func,text)==text:
        is_replaced=False
        return {'text':pattern.sub(return_func,text),'is_replaced':is_replaced}
    else:
        return {'text':pattern.sub(return_func,text),'is_replaced':is_replaced}
def splitArtist(artist):
    splitSymArtist=[' Featuring ',', ',' & ',' / ',' x ',' + ',' Duet With ',' X ',' With ',',']
    artist=re.split('|'.join(splitSymArtist),artist)[0]
    return artist

In [173]:
def getTunebat(title,artist):
    tunebatInfo=dict()
    notFoundTunebatInfo=dict()
    tunebat_url='https://tunebat.com/'
    keyword=(title+' '+splitArtist(artist)).replace(' ','+')
    status_code=0
    isResolved=False
    retry_c = 0
    while (status_code!=200 or isResolved==False) and retry_c < 10:
        r = requests.get(tunebat_url+'Search?q='+keyword
                         #, proxies=proxies
                        #,verify=False
                        )
        status_code = r.status_code
        if status_code != 200:
            retry_c += 1
            continue
        info_html=pq(r.text)
        info=info_html('script[src="/js/ReactUIComponents/SearchResults.jsx?v=1.10"]').next().text()#get data from script
        #clean text and to make it easier to operate, transform it info json format
        info=re.sub('\), document\.getElementById\("react_.*"\)\);','',re.sub(re.escape('ReactDOM.hydrate(React.createElement(SearchResultContainer, '),'',info))
        try:
            info=eval(info)
        except SyntaxError:
            retry_c +=1
            time.sleep(1)
            continue
        info=json.loads(info['data'])['TrackItems']
        if info!=[]:
            isResolved=True
            #info!=[]:
            info=info[0]# get first search result
            Key=info['Key']
            Camelot=info['Camelot']
            Duration=info['Duration']
            BPM=info['BPM']
            Popularity=info['Popularity']
            ReleaseDate=info['ReleaseDate']
            AlbumName=info['AlbumName']
            IsExplicit=info['IsExplicit']
            Label=info['Label']
            Energy=info['Energy']
            Danceability=info['Danceability']
            Happiness=info['Happiness']
            Loudness=info['Loudness']
            Acousticness=info['Acousticness']
            Instrumentalness=info['Instrumentalness']
            Liveness=info['Liveness']
            Speechiness=info['Speechiness']
            tunebatInfo=dict(zip([
            "Title","Artist","Key","Camelot","Duration","BPM","Popularity","ReleaseDate","AlbumName","IsExplicit","Label","Energy","Danceability","Happiness","Loudness","Acousticness","Instrumentalness","Liveness","Speechiness"
            ],
            [
                title, artist, Key, Camelot, Duration, BPM, Popularity, ReleaseDate, AlbumName, IsExplicit, Label, Energy, Danceability, Happiness, Loudness, Acousticness, Instrumentalness, Liveness, Speechiness
            ]))
        else:
            retry_c += 1
            time.sleep(1)
    if isResolved == False:      
        notFoundTunebatInfo=dict(zip(['Title','Artist'],[title,artist]))
    print('success\n',tunebat_url+'Search?q='+keyword+' has been processed.')
    return tunebatInfo,notFoundTunebatInfo



In [174]:
attrNameTunebatInfo=["Title","Artist","Key","Camelot","Duration","BPM","Popularity","ReleaseDate","AlbumName","IsExplicit","Label","Energy","Danceability","Happiness","Loudness","Acousticness","Instrumentalness","Liveness","Speechiness"]
attrNameNotFoundTunebatInfo=["Title","Artist"]
dfInfo= pd.DataFrame(columns=attrNameTunebatInfo)
dfNotFoundInfo= pd.DataFrame(columns=attrNameNotFoundTunebatInfo)
dataLists=list(csv.DictReader(open('song_rank_info.csv')))[0:2000]
ct=0
data=list()
dataNotFound=list()
for i in dataLists:
    tunebatInfo,notFoundTunebatInfo=getTunebat(i['title'],i['artist'])
    if tunebatInfo!={}:
        data.append(tunebatInfo)
    else:
        pass
    if notFoundTunebatInfo!={}:
        dataNotFound.append(notFoundTunebatInfo)
    else:
        pass
    ct+=1
    print('Total '+str(ct)+' Processed.')
for x in data:
    dfInfo=dfInfo.append(x,ignore_index=True)
for x in dataNotFound:
    dfNotFoundInfo=dfNotFoundInfo.append(x,ignore_index=True)
dfInfo.to_csv("info.csv", encoding = "UTF8")
dfNotFoundInfo.to_csv("not_found_info.csv", encoding = "UTF8")


success
 https://tunebat.com/Search?q=Who+I+Am+Hates+Who+I've+Been+Relient+K has been processed.
Total 1 Processed.
success
 https://tunebat.com/Search?q=Dare+Gorillaz has been processed.
Total 2 Processed.
success
 https://tunebat.com/Search?q=My+Hood+Young+Jeezy has been processed.
Total 3 Processed.
success
 https://tunebat.com/Search?q=The+Ghost+Of+You+My+Chemical+Romance has been processed.
Total 4 Processed.
success
 https://tunebat.com/Search?q=Temperature+Sean+Paul has been processed.
Total 5 Processed.
success
 https://tunebat.com/Search?q=Lean+Wit+It,+Rock+Wit+It+Dem+Franchize+Boyz has been processed.
Total 6 Processed.
success
 https://tunebat.com/Search?q=Ever+The+Same+Rob+Thomas has been processed.
Total 7 Processed.
success
 https://tunebat.com/Search?q=Get+Drunk+And+Be+Somebody+Toby+Keith has been processed.
Total 8 Processed.
success
 https://tunebat.com/Search?q=Walk+Away+Kelly+Clarkson has been processed.
Total 9 Processed.
success
 https://tunebat.com/Search?q=We+Bel

success
 https://tunebat.com/Search?q=Conceited+(There's+Something+About+Remy)+Remy+Ma has been processed.
Total 78 Processed.
success
 https://tunebat.com/Search?q=Wasteland+10+Years has been processed.
Total 79 Processed.
success
 https://tunebat.com/Search?q=Gettin'+Some+Shawnna has been processed.
Total 80 Processed.
success
 https://tunebat.com/Search?q=Gold+Lion+Yeah+Yeah+Yeahs has been processed.
Total 81 Processed.
success
 https://tunebat.com/Search?q=Beautiful+Love+The+Afters has been processed.
Total 82 Processed.
success
 https://tunebat.com/Search?q=A+Little+Less+Sixteen+Candles,+A+Little+More+Touch+Me+Fall+Out+Boy has been processed.
Total 83 Processed.
success
 https://tunebat.com/Search?q=The+Lucky+One+Faith+Hill has been processed.
Total 84 Processed.
success
 https://tunebat.com/Search?q=Snap+Yo+Fingers+Lil+Jon has been processed.
Total 85 Processed.
success
 https://tunebat.com/Search?q=Llame+Pa'+Verte+Wisin has been processed.
Total 86 Processed.
success
 https://tu

success
 https://tunebat.com/Search?q=I+Love+My+B****+Busta+Rhymes has been processed.
Total 154 Processed.
success
 https://tunebat.com/Search?q=Life+Is+A+Highway+Rascal+Flatts has been processed.
Total 155 Processed.
success
 https://tunebat.com/Search?q=Kick+Push+Lupe+Fiasco has been processed.
Total 156 Processed.
success
 https://tunebat.com/Search?q=One+Mary+J.+Blige+And+U2 has been processed.
Total 157 Processed.
success
 https://tunebat.com/Search?q=Shoulder+Lean+Young+Dro has been processed.
Total 158 Processed.
success
 https://tunebat.com/Search?q=I+Can't+Unlove+You+Kenny+Rogers has been processed.
Total 159 Processed.
success
 https://tunebat.com/Search?q=Ain't+No+Other+Man+Christina+Aguilera has been processed.
Total 160 Processed.
success
 https://tunebat.com/Search?q=Holla+At+Me+DJ+Khaled has been processed.
Total 161 Processed.
success
 https://tunebat.com/Search?q=Single+Natasha+Bedingfield has been processed.
Total 162 Processed.
success
 https://tunebat.com/Search?q=

success
 https://tunebat.com/Search?q=It's+Okay+(One+Blood)+The+Game has been processed.
Total 233 Processed.
success
 https://tunebat.com/Search?q=Too+Little+Too+Late+JoJo has been processed.
Total 234 Processed.
success
 https://tunebat.com/Search?q=Strut+The+Cheetah+Girls has been processed.
Total 235 Processed.
success
 https://tunebat.com/Search?q=Sleep+On+It+Danity+Kane has been processed.
Total 236 Processed.
success
 https://tunebat.com/Search?q=Ride+For+You+Danity+Kane has been processed.
Total 237 Processed.
success
 https://tunebat.com/Search?q=Feels+Just+Like+It+Should+Pat+Green has been processed.
Total 238 Processed.
success
 https://tunebat.com/Search?q=Mountains+Lonestar has been processed.
Total 239 Processed.
success
 https://tunebat.com/Search?q=The+Party's+Just+Begun+The+Cheetah+Girls has been processed.
Total 240 Processed.
success
 https://tunebat.com/Search?q=Morris+Brown+OutKast has been processed.
Total 241 Processed.
success
 https://tunebat.com/Search?q=Vans+

success
 https://tunebat.com/Search?q=It+Just+Comes+Natural+George+Strait has been processed.
Total 310 Processed.
success
 https://tunebat.com/Search?q=Watching+You+Rodney+Atkins has been processed.
Total 311 Processed.
success
 https://tunebat.com/Search?q=Upgrade+U+Beyonce has been processed.
Total 312 Processed.
success
 https://tunebat.com/Search?q=Dem+Jeans+Chingy has been processed.
Total 313 Processed.
success
 https://tunebat.com/Search?q=Change+Me+Ruben+Studdard has been processed.
Total 314 Processed.
success
 https://tunebat.com/Search?q=Crash+Here+Tonight+Toby+Keith has been processed.
Total 315 Processed.
success
 https://tunebat.com/Search?q=Here+(In+Your+Arms)+Hellogoodbye has been processed.
Total 316 Processed.
success
 https://tunebat.com/Search?q=Red+High+Heels+Kellie+Pickler has been processed.
Total 317 Processed.
success
 https://tunebat.com/Search?q=Tengo+Un+Amor+Toby+Love has been processed.
Total 318 Processed.
success
 https://tunebat.com/Search?q=Promise+Cia

...

success
 https://tunebat.com/Search?q=Let+It+Be+Kris+Allen has been processed.
Total 1626 Processed.
success
 https://tunebat.com/Search?q=Neighbors+Know+My+Name+Trey+Songz has been processed.
Total 1627 Processed.
success
 https://tunebat.com/Search?q=One+Day+Matisyahu has been processed.
Total 1628 Processed.
success
 https://tunebat.com/Search?q=Women+Lie,+Men+Lie+Yo+Gotti has been processed.
Total 1629 Processed.
success
 https://tunebat.com/Search?q=Gimmie+That+Girl+Joe+Nichols has been processed.
Total 1630 Processed.
success
 https://tunebat.com/Search?q=Backwoods+Justin+Moore has been processed.
Total 1631 Processed.
success
 https://tunebat.com/Search?q=Never+Let+You+Go+Justin+Bieber has been processed.
Total 1632 Processed.
success
 https://tunebat.com/Search?q=Ridin'+Solo+Jason+Derulo has been processed.
Total 1633 Processed.
success
 https://tunebat.com/Search?q=Lil+Freak+Usher has been processed.
Total 1634 Processed.
success
 https://tunebat.com/Search?q=Alice+Avril+Lavig

success
 https://tunebat.com/Search?q=Home+Glee+Cast has been processed.
Total 1704 Processed.
success
 https://tunebat.com/Search?q=Halfway+There+Big+Time+Rush has been processed.
Total 1705 Processed.
success
 https://tunebat.com/Search?q=Gettin'+Over+You+David+Guetta has been processed.
Total 1706 Processed.
success
 https://tunebat.com/Search?q=New+Morning+Alpha+Rev has been processed.
Total 1707 Processed.
success
 https://tunebat.com/Search?q=Not+Afraid+Eminem has been processed.
Total 1708 Processed.
success
 https://tunebat.com/Search?q=My+First+Kiss+3OH!3 has been processed.
Total 1709 Processed.
success
 https://tunebat.com/Search?q=Total+Eclipse+Of+The+Heart+Glee+Cast has been processed.
Total 1710 Processed.
success
 https://tunebat.com/Search?q=Find+Your+Love+Drake has been processed.
Total 1711 Processed.
success
 https://tunebat.com/Search?q=Lose+My+Mind+Young+Jeezy has been processed.
Total 1712 Processed.
success
 https://tunebat.com/Search?q=Run+Joey+Run+Glee+Cast has

success
 https://tunebat.com/Search?q=9+AM+In+Dallas+Drake has been processed.
Total 1782 Processed.
success
 https://tunebat.com/Search?q=Fireworks+Drake has been processed.
Total 1783 Processed.
success
 https://tunebat.com/Search?q=Another+Way+To+Die+Disturbed has been processed.
Total 1784 Processed.
success
 https://tunebat.com/Search?q=Double+Vision+3OH!3 has been processed.
Total 1785 Processed.
success
 https://tunebat.com/Search?q=Pretty+Boy+Swag+Soulja+Boy+Tell'em has been processed.
Total 1786 Processed.
success
 https://tunebat.com/Search?q=143+Bobby+Brackins has been processed.
Total 1787 Processed.
success
 https://tunebat.com/Search?q=Lay+Me+Down+The+Dirty+Heads has been processed.
Total 1788 Processed.
success
 https://tunebat.com/Search?q=Finding+My+Way+Back+Jaheim has been processed.
Total 1789 Processed.
success
 https://tunebat.com/Search?q=Farmer's+Daughter+Rodney+Atkins has been processed.
Total 1790 Processed.
success
 https://tunebat.com/Search?q=This+Ain't+Noth

success
 https://tunebat.com/Search?q=Last+Friday+Night+(T.G.I.F.)+Katy+Perry has been processed.
Total 1860 Processed.
success
 https://tunebat.com/Search?q=Toot+It+And+Boot+It+YG has been processed.
Total 1861 Processed.
success
 https://tunebat.com/Search?q=F**k+You!+(Forget+You)+Cee+Lo+Green has been processed.
Total 1862 Processed.
success
 https://tunebat.com/Search?q=Trailerhood+Toby+Keith has been processed.
Total 1863 Processed.
success
 https://tunebat.com/Search?q=Holding+You+Down+(Goin+In+Circles)+Jazmine+Sullivan has been processed.
Total 1864 Processed.
success
 https://tunebat.com/Search?q=Way+Out+Here+Josh+Thompson has been processed.
Total 1865 Processed.
success
 https://tunebat.com/Search?q=The+Breath+You+Take+George+Strait has been processed.
Total 1866 Processed.
success
 https://tunebat.com/Search?q=Only+Prettier+Miranda+Lambert has been processed.
Total 1867 Processed.
success
 https://tunebat.com/Search?q=Anything+Like+Me+Brad+Paisley has been processed.
Total 1

success
 https://tunebat.com/Search?q=This+Ain't+No+Love+Song+Trace+Adkins has been processed.
Total 1936 Processed.
success
 https://tunebat.com/Search?q=Mean+Taylor+Swift has been processed.
Total 1937 Processed.
success
 https://tunebat.com/Search?q=Firework+Katy+Perry has been processed.
Total 1938 Processed.
success
 https://tunebat.com/Search?q=Gonna+Get+This+Hannah+Montana has been processed.
Total 1939 Processed.
success
 https://tunebat.com/Search?q=Whip+My+Hair+WILLOW has been processed.
Total 1940 Processed.
success
 https://tunebat.com/Search?q=Monster+Kanye+West has been processed.
Total 1941 Processed.
success
 https://tunebat.com/Search?q=The+End+Kings+Of+Leon has been processed.
Total 1942 Processed.
success
 https://tunebat.com/Search?q=What's+My+Name?+Rihanna has been processed.
Total 1943 Processed.
success
 https://tunebat.com/Search?q=Rap+Song+T-Pain has been processed.
Total 1944 Processed.
success
 https://tunebat.com/Search?q=Porn+Star+Dancing+My+Darkest+Days ha

In [178]:
dfInfo[]

,Title,Artist,Key,Camelot,Duration,BPM,Popularity,ReleaseDate,AlbumName,IsExplicit,Label,Energy,Danceability,Happiness,Loudness,Acousticness,Instrumentalness,Liveness,Speechiness
0,Who I Am Hates Who I've Been,Relient K,E♭ Major,5B,3:52,167,52,NA,Mmhmm,False,NA,90,40,49,-4 dB,0,0,7,9
1,Dare,"Gorillaz,Shaun Ryder",B Minor,10A,4:04,120,70,NA,Demon Days,False,NA,89,76,97,-6 dB,2,9,30,4
2,My Hood,Young Jeezy,D♭ Minor,12A,4:00,152,46,NA,Let's Get It: Thug Motivation 101,True,NA,68,83,76,-7 dB,2,0,4,8
3,The Ghost Of You,My Chemical Romance,B Minor,10A,3:14,146,64,NA,Three Cheers for Sweet Revenge,False,NA,89,20,19,-4 dB,3,0,64,8
4,Temperature,Sean Paul,C Minor,5A,3:38,125,75,NA,The Trinity,False,NA,60,95,82,-5 dB,11,0,7,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1923,Liv Tonight,"Nelly,Keri Hilson",A♭ Major,4B,4:31,135,31,NA,5.0,True,NA,91,59,65,-5 dB,0,0,5,9
1924,Hold My Hand,Michael Jackson Duet With Akon,D♭ Major,3B,3:32,90,60,NA,Michael,False,NA,72,61,38,-6 dB,19,0,11,4
1925,Marry Me,Train,C Major,8B,3:25,88,71,NA,"Save Me, San Francisco (Golden Gate Edition)",False,NA,27,46,48,-9 dB,83,0,11,3
1926,Blow,Ke$ha,B Minor,10A,3:39,120,61,NA,Cannibal (Expanded Edition),False,NA,73,75,81,-4 dB,0,0,7,4


In [187]:
attrNameTunebatInfo=["Title","Artist","Key","Camelot","Duration","BPM","Popularity","ReleaseDate","AlbumName","IsExplicit","Label","Energy","Danceability","Happiness","Loudness","Acousticness","Instrumentalness","Liveness","Speechiness"]
attrNameNotFoundTunebatInfo=["Title","Artist"]
dfInfo_ex= pd.DataFrame(columns=attrNameTunebatInfo)
dfNotFoundInfo_ex= pd.DataFrame(columns=attrNameNotFoundTunebatInfo)
dataLists=list(csv.DictReader(open('not_found_info.csv')))
ct=0
data_ex=list()
dataNotFound_ex=list()
for i in dataLists:
    tunebatInfo,notFoundTunebatInfo=getTunebat(i['Title'],i['Artist'])
    if tunebatInfo!={}:
        data_ex.append(tunebatInfo)
    else:
        pass
    if notFoundTunebatInfo!={}:
        dataNotFound_ex.append(notFoundTunebatInfo)
    else:
        pass
    ct+=1
    print('Total '+str(ct)+' Processed.')
for x in data_ex:
    dfInfo_ex=dfInfo_ex.append(x,ignore_index=True)
for x in dataNotFound_ex:
    dfNotFoundInfo_ex=dfNotFoundInfo_ex.append(x,ignore_index=True)
dfInfo_ex.to_csv("info_ex.csv", encoding = "UTF8")
dfNotFoundInfo_ex.to_csv("not_found_info_ex.csv", encoding = "UTF8")

success
 https://tunebat.com/Search?q=Get'cha+Head+In+The+Game+Andrew+Seeley has been processed.
Total 1 Processed.
success
 https://tunebat.com/Search?q=Start+Of+Something+New+Zac+Efron has been processed.
Total 2 Processed.
success
 https://tunebat.com/Search?q=What+I've+Been+Looking+For+(Reprise)+Andrew+Seeley has been processed.
Total 3 Processed.
success
 https://tunebat.com/Search?q=When+There+Was+Me+And+You+Vanessa+Anne+Hudgens has been processed.
Total 4 Processed.
success
 https://tunebat.com/Search?q=In+The+Sun+Michael+Stipe has been processed.
Total 5 Processed.
success
 https://tunebat.com/Search?q=Soundtrack+To+Your+Life+Ashley+Parker+Angel has been processed.
Total 6 Processed.
success
 https://tunebat.com/Search?q=Holla+At+Me+DJ+Khaled has been processed.
Total 7 Processed.
success
 https://tunebat.com/Search?q=Love+Me+Or+Hate+Me+(F**k+You!!!!)+Lady+Sovereign has been processed.
Total 8 Processed.
success
 https://tunebat.com/Search?q=Kingdom+Come+JAY-Z has been processe

success
 https://tunebat.com/Search?q=Boyfriend+New+Hollow has been processed.
Total 75 Processed.
success
 https://tunebat.com/Search?q=Hate+Sleeping+Alone+Drake has been processed.
Total 76 Processed.
success
 https://tunebat.com/Search?q=We've+Got+Tonight+Phillip+Phillips has been processed.
Total 77 Processed.
success
 https://tunebat.com/Search?q=Lover's+Eyes+Mumford has been processed.
Total 78 Processed.
success
 https://tunebat.com/Search?q=#thatPOWER+will.i.am has been processed.
Total 79 Processed.
success
 https://tunebat.com/Search?q=#Beautiful+Mariah+Carey has been processed.
Total 80 Processed.
success
 https://tunebat.com/Search?q=Crown+JAY-Z has been processed.
Total 81 Processed.
success
 https://tunebat.com/Search?q=Chinese+Food+Alison+Gold has been processed.
Total 82 Processed.
success
 https://tunebat.com/Search?q=Angel+Jacquie+Lee has been processed.
Total 83 Processed.
success
 https://tunebat.com/Search?q=#SELFIE+The+Chainsmokers has been processed.
Total 84 Pro

In [186]:
dfInfo_ex

,Title,Artist,Key,Camelot,Duration,BPM,Popularity,ReleaseDate,AlbumName,IsExplicit,Label,Energy,Danceability,Happiness,Loudness,Acousticness,Instrumentalness,Liveness,Speechiness
0,Who I Am Hates Who I've Been,Relient K,E♭ Major,5B,3:52,167,52,NA,Mmhmm,False,NA,90,40,49,-4 dB,0,0,7,9
1,Dare,"Gorillaz,Shaun Ryder",B Minor,10A,4:04,120,70,NA,Demon Days,False,NA,89,76,97,-6 dB,2,9,30,4
2,My Hood,Young Jeezy,D♭ Minor,12A,4:00,152,46,NA,Let's Get It: Thug Motivation 101,True,NA,68,83,76,-7 dB,2,0,4,8
3,The Ghost Of You,My Chemical Romance,B Minor,10A,3:14,146,64,NA,Three Cheers for Sweet Revenge,False,NA,89,20,19,-4 dB,3,0,64,8
4,Temperature,Sean Paul,C Minor,5A,3:38,125,75,NA,The Trinity,False,NA,60,95,82,-5 dB,11,0,7,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1923,Liv Tonight,"Nelly,Keri Hilson",A♭ Major,4B,4:31,135,31,NA,5.0,True,NA,91,59,65,-5 dB,0,0,5,9
1924,Hold My Hand,Michael Jackson Duet With Akon,D♭ Major,3B,3:32,90,60,NA,Michael,False,NA,72,61,38,-6 dB,19,0,11,4
1925,Marry Me,Train,C Major,8B,3:25,88,71,NA,"Save Me, San Francisco (Golden Gate Edition)",False,NA,27,46,48,-9 dB,83,0,11,3
1926,Blow,Ke$ha,B Minor,10A,3:39,120,61,NA,Cannibal (Expanded Edition),False,NA,73,75,81,-4 dB,0,0,7,4
